In [28]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)


2.0.0


In [62]:
# https://www.tensorflow.org/tutorials/keras/regression
years = [
    '200001', '200102', '200203', '200304', '200405', '200506', '200607', '200708', '200809', '200910',
    '201011', '201112', '201213','201314', '201415','201516', '201617', '201718'
]
train_dataset = None
for year in years: 
#     year= '200001'
    path = './data/formatted/playoff/playoffdata' + year + '.csv'
    raw_dataset = pd.read_csv(path)
    if train_dataset is None:
        train_dataset = raw_dataset.copy()
    else: 
        train_dataset = pd.concat([train_dataset,raw_dataset.copy()])

test_dataset = pd.read_csv('./data/formatted/playoff/playoffdata201819.csv')

# Plot data using seaborn
# sns.pairplot(dataset[["winnerSeed", "loserSeed"]], diag_kind="kde")

# Separate the target value from the feature. This label is the value that you will train the model to predict
train_labels = train_dataset.pop('ID')
train_dataset.pop('ID.1')
print(len(train_dataset.keys()))

# Look at overall statistic:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()


52


In [61]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
print(len(normed_train_data.keys()))
normed_test_data = norm(test_dataset)


52


In [63]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()

# Inspect model
model.summary()

example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
# example_result

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 64)                3392      
_________________________________________________________________
dense_31 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 65        
Total params: 7,617
Trainable params: 7,617
Non-trainable params: 0
_________________________________________________________________


In [64]:
### TRAIN MODEL
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])



....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........................................................................................

In [66]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.head()


,loss,mae,mse,val_loss,val_mae,val_mse,epoch
0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,4
